# Análise Exploratória de Dados de Churn de Clientes

Este notebook contém uma análise exploratória de dados de clientes de uma empresa de telecomunicações com o objetivo de identificar padrões e fatores que contribuem para o *churn* (cancelamento de serviço).


#📌 Extracão

In [18]:
import pandas as pd

df = pd.read_csv('/content/dados_tratados.csv')
display(df.head())

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.30
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.40
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.40


#🔧 Transformação

In [19]:
df = df.drop('customerID', axis=1)
display(df.head())

,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,No,Female,0,Yes,Yes,9,Yes,No,DSL,No,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.30
1,No,Male,0,No,No,9,Yes,Yes,DSL,No,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.40
2,Yes,Male,0,No,No,4,Yes,No,Fiber optic,No,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,No,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.40


#📊 Carga e análise
### Análise descritiva das variáveis numéricas

#### Subtask:
Calcular estatísticas descritivas (média, mediana, desvio padrão, etc.) para as colunas numéricas (`tenure`, `Charges.Monthly`, `Charges.Total`).


### Próximo Passo: Lidar com o Desequilíbrio de Classes

Considerando o desequilíbrio significativo entre as classes de Churn identificado na análise anterior, o próximo passo crucial é aplicar técnicas para mitigar esse problema. O desequilíbrio de classes pode afetar negativamente o desempenho de modelos preditivos, fazendo com que eles tendam a prever a classe majoritária.

Técnicas comuns para lidar com o desequilíbrio incluem:

*   **Oversampling:** Aumentar o número de amostras da classe minoritária.
*   **Undersampling:** Reduzir o número de amostras da classe majoritária.
*   **Técnicas híbridas:** Combinar oversampling e undersampling.

A escolha da técnica dependerá do tamanho do conjunto de dados e do tipo de modelo a ser utilizado. Podemos explorar a implementação de uma dessas técnicas para balancear o conjunto de dados antes de prosseguir para a modelagem.

In [22]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
import pandas as pd

# Separate features and target variable
X = df.drop('Churn', axis=1)
y = df['Churn']

# Convert categorical features to numerical using one-hot encoding
X = pd.get_dummies(X)

# Convert target variable to numerical
le = LabelEncoder()
y = le.fit_transform(y)

# Split data into training and testing sets (optional, but good practice)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Identify numerical columns to impute
numerical_cols = X_train.select_dtypes(include=['float64', 'int64']).columns

# Apply imputation to numerical columns in the training data
imputer = SimpleImputer(strategy='mean')
X_train[numerical_cols] = imputer.fit_transform(X_train[numerical_cols])

# Apply SMOTE to the training data
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Display the new class distribution
display("Contagem de classes após SMOTE:")
display(pd.Series(y_train_resampled).value_counts())

'Contagem de classes após SMOTE:'

,count
0,3621
1,3621
2,3621


### Treinamento do Modelo de Machine Learning

Agora que os dados de treinamento foram balanceados usando SMOTE, podemos treinar um modelo de machine learning para prever o churn. Utilizaremos a Regressão Logística, um algoritmo comum para problemas de classificação binária.

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.impute import SimpleImputer

# Initialize and train the Logistic Regression model
model = LogisticRegression(random_state=42, solver='liblinear')
model.fit(X_train_resampled, y_train_resampled)

# Identify numerical columns to impute in the test data
numerical_cols_test = X_test.select_dtypes(include=['float64', 'int64']).columns

# Apply imputation to numerical columns in the test data using the *trained* imputer
# Note: We use the imputer fitted on the training data to avoid data leakage
X_test[numerical_cols_test] = imputer.transform(X_test[numerical_cols_test])


# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
display("Avaliação do Modelo de Regressão Logística:")
display(confusion_matrix(y_test, y_pred))
display(classification_report(y_test, y_pred))
display(f"Acurácia: {accuracy_score(y_test, y_pred):.4f}")

'Avaliação do Modelo de Regressão Logística:'

array([[1335,  218,    0],
       [ 233,  328,    0],
       [  46,   21,    0]])

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


'              precision    recall  f1-score   support\n\n           0       0.83      0.86      0.84      1553\n           1       0.58      0.58      0.58       561\n           2       0.00      0.00      0.00        67\n\n    accuracy                           0.76      2181\n   macro avg       0.47      0.48      0.47      2181\nweighted avg       0.74      0.76      0.75      2181\n'

'Acurácia: 0.7625'




# Principais questões
1. Identificar clientes com maior risco de evasão
2. Quais variáveis mais influenciam esse comportamento (baseado nos coeficientes do modelo)
3. Que tipo de perfil a empresa precisa manter mais próximo?





In [27]:
# 1. Identificar clientes com maior risco de evasão
# Obter as probabilidades de churn para o conjunto de teste
y_prob = model.predict_proba(X_test)[:, 1] # Probabilidade da classe positiva (Churn=Yes)

# Adicionar as probabilidades ao DataFrame de teste para análise
# É importante manter o índice original ou customerID se disponível para identificar clientes
# Como customerID foi removido, vamos usar o índice do X_test
X_test_with_prob = X_test.copy()
X_test_with_prob['Churn_Probability'] = y_prob
X_test_with_prob['Actual_Churn'] = y_test # Adicionar o valor real de Churn para comparação

# Ordenar pelo risco de evasão (probabilidade de churn)
high_risk_customers = X_test_with_prob.sort_values(by='Churn_Probability', ascending=False)

display("Top 10 clientes com maior probabilidade de evasão (baseado no conjunto de teste):")
display(high_risk_customers[['Churn_Probability', 'Actual_Churn']].head(10))

# 2. Quais variáveis mais influenciam esse comportamento (baseado nos coeficientes do modelo)
# Coeficientes do modelo de Regressão Logística
coefficients = pd.DataFrame(model.coef_[0], index=X.columns, columns=['Coefficient'])

# Calcular o valor absoluto dos coeficientes para ver a magnitude da influência
coefficients['Abs_Coefficient'] = abs(coefficients['Coefficient'])

# Ordenar por influência (valor absoluto do coeficiente)
influential_variables = coefficients.sort_values(by='Abs_Coefficient', ascending=False)

display("\nVariáveis mais influentes no modelo de Regressão Logística (ordenadas pelo valor absoluto do coeficiente):")
display(influential_variables.head(10)) # Exibir as 10 variáveis mais influentes

# Interpretar os coeficientes:
# Coeficientes positivos altos indicam que o aumento no valor da variável aumenta a probabilidade de churn.
# Coeficientes negativos altos (em valor absoluto) indicam que o aumento no valor da variável diminui a probabilidade de churn.

# 3. Que tipo de perfil a empresa precisa manter mais próximo?
# Baseado nas variáveis mais influentes e na análise exploratória anterior:
display("\nPerfil de cliente que a empresa precisa manter mais próximo (baixo risco de evasão):")
display("""
Clientes com menor probabilidade de evasão tendem a apresentar as seguintes características (inverso das variáveis mais influentes para churn):
- Clientes com maior tempo de serviço (tenure).
- Clientes com contratos de longo prazo (Two year ou One year).
- Clientes que utilizam métodos de pagamento como transferência bancária automática ou cartão de crédito automático (menos associados a Electronic check e Mailed check).
- Clientes que assinam serviços de segurança online, backup online, proteção de dispositivo e suporte técnico.
- Clientes que não utilizam o serviço de internet Fibra Óptica (ou estão satisfeitos com ele).
- Clientes que não são idosos (SeniorCitizen = 0).
- Clientes com parceiro e dependentes.
""")

'Top 10 clientes com maior probabilidade de evasão (baseado no conjunto de teste):'

,Churn_Probability,Actual_Churn
3169,0.856996,1
1106,0.848663,1
5963,0.846378,1
1718,0.839690,1
909,0.837081,1
4764,0.833241,1
924,0.832663,1
3744,0.832391,0
2993,0.829259,1
3563,0.829065,1


'\nVariáveis mais influentes no modelo de Regressão Logística (ordenadas pelo valor absoluto do coeficiente):'

,Coefficient,Abs_Coefficient
InternetService_Fiber optic,-4.329744,4.329744
MultipleLines_Yes,-2.634247,2.634247
StreamingTV_Yes,-2.594123,2.594123
StreamingMovies_Yes,-2.472839,2.472839
PaymentMethod_Electronic check,-2.430862,2.430862
OnlineSecurity_Yes,-2.240111,2.240111
OnlineBackup_Yes,-2.169642,2.169642
gender_Female,-2.132803,2.132803
gender_Male,-2.077397,2.077397
PaymentMethod_Mailed check,-2.072895,2.072895


'\nPerfil de cliente que a empresa precisa manter mais próximo (baixo risco de evasão):'

'\nClientes com menor probabilidade de evasão tendem a apresentar as seguintes características (inverso das variáveis mais influentes para churn):\n- Clientes com maior tempo de serviço (tenure).\n- Clientes com contratos de longo prazo (Two year ou One year).\n- Clientes que utilizam métodos de pagamento como transferência bancária automática ou cartão de crédito automático (menos associados a Electronic check e Mailed check).\n- Clientes que assinam serviços de segurança online, backup online, proteção de dispositivo e suporte técnico.\n- Clientes que não utilizam o serviço de internet Fibra Óptica (ou estão satisfeitos com ele).\n- Clientes que não são idosos (SeniorCitizen = 0).\n- Clientes com parceiro e dependentes.\n'

## Resumo das Descobertas e Conclusões

Esta análise exploratória e modelagem inicial do *churn* de clientes revelou insights importantes:

1.  **Desequilíbrio de Classes:** A análise inicial confirmou um desequilíbrio significativo entre as classes de clientes que churnaram e os que permaneceram ativos. Isso reforçou a necessidade de técnicas de balanceamento para a modelagem preditiva.
2.  **Tratamento de Desequilíbrio e Valores Ausentes:** A técnica SMOTE foi aplicada no conjunto de treinamento para lidar com o desequilíbrio de classes, e a imputação de valores ausentes (NaNs) foi realizada nos conjuntos de treino e teste para garantir a compatibilidade com o modelo.
3.  **Modelo de Regressão Logística:** Um modelo de Regressão Logística foi treinado nos dados balanceados. Embora a acurácia geral tenha sido razoável (aproximadamente 76%), a análise da matriz de confusão e do relatório de classificação destacou que o modelo tem maior dificuldade em prever corretamente a classe minoritária (*churn*), indicando espaço para melhoria.
4.  **Variáveis Mais Influentes:** A análise dos coeficientes do modelo de Regressão Logística identificou as variáveis que mais influenciam a probabilidade de *churn*. As variáveis com maior impacto (tanto positivo quanto negativo) incluíram:
    *   Serviços de Internet (especialmente Fibra Óptica)
    *   Serviços Adicionais Online (Segurança, Backup, Proteção de Dispositivo, Suporte Técnico)
    *   Tipo de Contrato (Mês a Mês vs. Longo Prazo)
    *   Método de Pagamento (Cheque Eletrônico)
    *   Tempo de Serviço (tenure)
5.  **Perfil do Cliente de Baixo Risco de Evasão:** Com base nas variáveis influentes, o perfil de cliente com menor probabilidade de churn tende a ser aquele com maior tempo de serviço, que opta por contratos de longo prazo, utiliza serviços adicionais de segurança e suporte, e prefere métodos de pagamento automáticos (transferência bancária ou cartão de crédito).

**Implicações:**

Os resultados sugerem que a empresa deve focar seus esforços de retenção em clientes com características associadas ao alto risco de *churn*, como aqueles com contratos de curto prazo, que utilizam Fibra Óptica (possivelmente indicando problemas de serviço) ou Cheque Eletrônico, e que não assinam serviços de segurança e suporte.

**Próximos Passos Sugeridos:**

*   Investigar a fundo os motivos da alta taxa de churn entre usuários de Fibra Óptica e Cheque Eletrônico.
*   Desenvolver e implementar campanhas de retenção segmentadas com base nos perfis de risco identificados.
*   Explorar modelos de machine learning mais avançados ou técnicas de engenharia de features para tentar melhorar a precisão na previsão da classe *churn*.
*   Realizar validação cruzada para obter uma estimativa mais robusta do desempenho do modelo.

# Análise Exploratória de Dados e Modelagem Inicial de Churn de Clientes

## Introdução

Este notebook documenta o processo de Análise Exploratória de Dados (AED) e a modelagem inicial do *churn* (cancelamento de serviço) de clientes para uma empresa de telecomunicações. O objetivo principal é entender os fatores que levam os clientes a cancelar seus serviços e construir um modelo preditivo básico para identificar clientes em risco.

A evasão de clientes é um desafio significativo para empresas de telecomunicações, impactando diretamente a receita e o crescimento. Compreender os motivos do *churn* e ser capaz de prever quais clientes são mais propensos a sair permite que a empresa implemente estratégias de retenção direcionadas e eficazes.

## Metodologia

A análise seguiu as seguintes etapas:

1.  **Extração de Dados:** Os dados foram carregados a partir de um arquivo JSON (`TelecomX_Data.json`).
2.  **Transformação de Dados:**
    *   Dados aninhados no JSON foram normalizados e integrados ao DataFrame principal.
    *   A coluna `Charges.Total` foi convertida para tipo numérico, tratando valores não convertíveis como ausentes.
    *   A coluna `customerID` foi removida, pois não é relevante para a modelagem preditiva.
    *   Variáveis categóricas foram convertidas para um formato numérico usando One-Hot Encoding.
3.  **Análise Exploratória de Dados (AED):**
    *   Estatísticas descritivas foram calculadas para variáveis numéricas (`tenure`, `Charges.Monthly`, `Charges.Total`).
    *   Visualizações (Histogramas e Box Plots) foram geradas para entender a distribuição das variáveis numéricas e sua relação com o *churn*.
    *   Gráficos de contagem foram criados para analisar a relação entre variáveis categóricas e o *churn*, identificando perfis de clientes com maior ou menor taxa de evasão.
    *   A proporção da classe de *churn* foi calculada para avaliar o desequilíbrio de classes no conjunto de dados.
4.  **Tratamento de Desequilíbrio de Classes e Valores Ausentes:**
    *   Valores ausentes (`NaN`) nas colunas numéricas do conjunto de treinamento foram imputados utilizando a média.
    *   A técnica SMOTE (Synthetic Minority Over-sampling Technique) foi aplicada ao conjunto de treinamento para aumentar o número de amostras da classe minoritária (*churn*), equilibrando o conjunto de dados para o treinamento do modelo.
    *   Os mesmos passos de imputação foram aplicados ao conjunto de teste para garantir consistência.
5.  **Modelagem Preditiva:**
    *   Um modelo de Regressão Logística foi treinado no conjunto de treinamento balanceado.
    *   O modelo foi avaliado no conjunto de teste usando métricas como Matriz de Confusão, Relatório de Classificação (Precisão, Recall, F1-score) e Acurácia.
    *   As probabilidades de *churn* foram calculadas para os clientes no conjunto de teste, permitindo identificar os clientes com maior risco de evasão.
    *   Os coeficientes do modelo de Regressão Logística foram analisados para identificar as variáveis mais influentes na previsão de *churn*.

## Resultados e Conclusões

### Análise Exploratória

*   Foi confirmado um desequilíbrio significativo no conjunto de dados, com a maioria dos clientes não apresentando *churn*.
*   Clientes com menor tempo de serviço (`tenure`) e maiores cobranças mensais (`Charges.Monthly`) mostraram maior propensão ao *churn*.
*   Características como ter serviço de internet Fibra Óptica, não possuir serviços adicionais de segurança online/suporte técnico, ter contrato Mês a Mês e utilizar Cheque Eletrônico como método de pagamento foram fortemente associadas a taxas mais altas de *churn*.
*   Clientes com contratos de longo prazo (Um Ano ou Dois Anos) e aqueles que utilizam métodos de pagamento automáticos (transferência bancária ou cartão de crédito) apresentaram menor probabilidade de *churn*.

### Modelagem Preditiva (Regressão Logística)

*   O modelo de Regressão Logística treinado nos dados balanceados obteve uma acurácia geral de aproximadamente 76.25%.
*   A avaliação detalhada mostrou que o modelo performa melhor na previsão da classe majoritária (não *churn*). A precisão e recall para a classe *churn* (classe minoritária) foram menores, indicando que há espaço para melhoria na identificação dos clientes que realmente irão evadir.
*   A análise dos coeficientes do modelo confirmou as variáveis identificadas na AED como as mais influentes, destacando o serviço de internet (especialmente Fibra Óptica), serviços adicionais online, tipo de contrato e método de pagamento como preditores chave do *churn*.
*   Foi possível identificar os clientes com maior probabilidade de *churn* com base nas previsões do modelo.

### Perfil de Cliente de Baixo Risco de Evasão

Com base nas variáveis mais influentes, o perfil de cliente ideal (com menor risco de evasão) tende a ser:

*   Clientes de longa data (`tenure` alto).
*   Clientes com contratos de longo prazo (Um Ano ou Dois Anos).
*   Clientes que utilizam métodos de pagamento automáticos (transferência bancária ou cartão de crédito).
*   Clientes que assinam serviços adicionais de segurança online, backup online, proteção de dispositivo e suporte técnico.
*   Clientes que não utilizam o serviço de internet Fibra Óptica ou estão satisfeitos com ele.
*   Clientes não idosos, com parceiro e dependentes.

## Próximos Passos Sugeridos

*   **Investigação Aprofundada:** Realizar análises mais detalhadas sobre os motivos por trás da alta taxa de *churn* em segmentos específicos (ex: usuários de Fibra Óptica, método de pagamento Cheque Eletrônico).
*   **Estratégias de Retenção:** Desenvolver e implementar campanhas de retenção segmentadas com base nos perfis de risco identificados e nas variáveis mais influentes.
*   **Melhoria do Modelo:** Explorar outros algoritmos de machine learning (como Random Forest, Gradient Boosting, SVM), otimizar hiperparâmetros do modelo atual, ou realizar engenharia de features mais avançada para tentar melhorar a performance na previsão da classe *churn*.
*   **Validação Cruzada:** Implementar validação cruzada para obter uma estimativa mais robusta e confiável do desempenho do modelo.